In [126]:
import sys
print(sys.version)

3.12.11 (main, Jun  3 2025, 15:41:47) [Clang 17.0.0 (clang-1700.0.13.3)]


In [127]:
%load_ext autoreload
%autoreload 0

import sys
import os
import json
from pprint import pprint
from operator import itemgetter

from draftfast import rules
from draftfast.optimize import run
from draftfast.orm import Player
from draftfast.csv_parse import salary_download
from draftfast.settings import OptimizerSettings, CustomRule, PlayerPoolSettings
from draftfast.lineup_constraints import LineupConstraints
import pandas as pd
import numpy as np
import math
from datetime import date, datetime, timedelta
from scipy import stats
from nfl_teams import NFL_TEAM_MAP
from odds import get_metabet_spread, get_current_rankings, get_fantasy_def_points_against
from collections import defaultdict

def get_most_recently_created_file_with_extension(folder, extension):
    files = [f for f in os.listdir(folder) if f.endswith(extension)]
    return max(files, key=lambda x: os.path.getctime(f"{folder}/{x}"))

# https://www.pro-football-reference.com/years/2021/fantasy.htm
DATA_FOLDER = './data25'
ACTIVE_FOLDER = './active'
UPLOAD_FOLDER = './upload'
SALARY_FILE = f"{DATA_FOLDER}/{get_most_recently_created_file_with_extension(DATA_FOLDER, 'csv')}"
SEASON_START = '09/02/2025'

def get_week_relative_to_start(season_start=SEASON_START):
    start = datetime.strptime(season_start, '%m/%d/%Y')
    today = datetime.today()
    num_days = (today - start).days + 1
    return math.ceil(num_days / 7)


WEEK = max(get_week_relative_to_start(), 1)

WEIGHTED = True

ACTIVE_FILE = f"{ACTIVE_FOLDER}/data.csv"
UPLOAD_FILE = f"{UPLOAD_FOLDER}/upload.csv"
MAX_PLAYED = WEEK - 1

df = pd.read_csv(SALARY_FILE, na_values= '')

print('ready', SALARY_FILE, WEEK)
# print(df.head())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
ready ./data25/FanDuel-NFL-2025 EST-12 EST-22 EST-124667-players-list (1).csv 16


In [128]:
# Check df immediately after loading
print(f"\nDataFrame loaded with {df.shape[0]} rows and {df.shape[1]} columns")
print(f"Positions: {df['Position'].unique()}")
print(f"Position value counts:\n{df['Position'].value_counts()}")



DataFrame loaded with 56 rows and 18 columns
Positions: ['RB' 'QB' 'TE' 'WR' 'D' 'K']
Position value counts:
Position
WR    20
RB    12
TE    11
QB     8
K      3
D      2
Name: count, dtype: int64


In [129]:
# Manual step: Load NFL team rankings from a local CSV or DataFrame instead of get_current_rankings.
# If you do not have a rankings file, this cell will safely skip and the rest of the notebook will work without rankings.
import os
import pandas as pd
RANKINGS_CSV = './ranking/defense_1.json'  # Update this path to your rankings file (CSV or JSON)
if os.path.isfile(RANKINGS_CSV):
    try:
        if RANKINGS_CSV.endswith('.csv'):
            ranking_df = pd.read_csv(RANKINGS_CSV)
        elif RANKINGS_CSV.endswith('.json'):
            ranking_df = pd.read_json(RANKINGS_CSV)
        else:
            raise ValueError('Unsupported file format for rankings.')
    except Exception as e:
        print('Error loading rankings:', e)
        ranking_df = pd.DataFrame()  # Empty fallback
else:
    print(f'Rankings file not found: {RANKINGS_CSV}. Proceeding without rankings.')
    ranking_df = pd.DataFrame()  # Empty fallback
    # You can also paste your DataFrame here manually if needed.

def get_abbr(x):
    try:
        short_team = NFL_TEAM_MAP[x]
        print(x, short_team)
        return short_team
    except Exception as e:
        print(e, x)

if not ranking_df.empty and 'team_fk__full_name' in ranking_df.columns:
    ranking_df['team'] = ranking_df['team_fk__full_name'].apply(get_abbr)
else:
    print('Ranking DataFrame is empty or missing expected columns.')
# DEF_KEYS = ['passing_interceptions_rank'
# ranking_df['def_rank'] = ranking_df.apply(lambda row: np.avg([row[k] for k in DEF_KEYS]))
try:
    if 'df' in globals() and 'Team' in df.columns:
        num_teams = df['Team'].nunique()
    else:
        num_teams = 0
except Exception as e:
    print('Error determining number of teams from data CSV:', e)
    num_teams = 0

ranking_df.shape
print(ranking_df.columns.values)

Rankings file not found: ./ranking/defense_1.json. Proceeding without rankings.
Ranking DataFrame is empty or missing expected columns.
[]


In [130]:
ranking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


In [131]:
# rankings = ranking_df.to_dict('records') if not ranking_df.empty else []
if ranking_df is not None and not ranking_df.empty and 'team' in ranking_df.columns:
    rankings = ranking_df.to_dict('records')
    print(rankings[0]['team'])
    rankings = {x['team']: x for x in  rankings}
    teams = set(rankings.keys())
    print(len(teams), teams)
else:
    print('No rankings data loaded. Proceeding without rankings.')
    rankings = {}
    teams = set()

No rankings data loaded. Proceeding without rankings.


In [132]:
spread_df = get_metabet_spread(WEEK)
favor_map = {}
z_map = {}
z_scores = {}
print(spread_df)

return cached data week 16
    Unnamed: 0 AwayTeam HomeTeam  PointSpread  OverUnder
0            0      LAR      SEA        -2.19      42.44
1            1      PHI      WAS         7.08      43.50
2            2       GB      CHI        -1.43      44.85
3            3       NE      BAL        -3.00      48.63
4            4      BUF      CLE        10.44      41.10
5            5      LAC      DAL        -1.11      50.52
6            6      NYJ       NO        -6.56      40.27
7            7       TB      CAR         2.98      44.60
8            8      MIN      NYG         2.50      41.48
9            9       KC      TEN         3.07      37.50
10          10      CIN      MIA         3.56      48.04
11          11      JAC      DEN        -3.43      46.72
12          12      ATL      ARI         2.68      47.68
13          13       LV      HOU       -14.24      39.44
14          14      PIT      DET        -7.58      51.98
15          15       SF      IND         5.56      46.00


In [133]:


if 'OverUnder' in spread_df.columns.values:
    points = list(spread_df['OverUnder'])
    zs = stats.zscore(points)
    for i, p in enumerate(points):
        z_scores[p] = 0 if np.isnan(zs[i]) else zs[i]

# Vegas odds convention: PointSpread is from the home team's perspective
# Negative spread = home team is favored by that amount
# Positive spread = away team is favored by that amount
# For favor_map: positive = team is unfavored, negative = team is favored
for index, row in spread_df.iterrows():
    home, away = row['HomeTeam'], row['AwayTeam']
    # PointSpread represents home team's advantage
    # Direct mapping: home team gets the spread as-is, away team gets negated
    favor_map[home] = row['PointSpread']   # Home team (negative = favored, positive = unfavored)
    favor_map[away] = -row['PointSpread']  # Away team (opposite of home)
    if 'OverUnder' in row:
        z_map[home] = z_scores[row['OverUnder']]
        z_map[away] = z_scores[row['OverUnder']]



if 'JAX' in favor_map:
    favor_map['JAC'] = favor_map['JAX']
    z_map['JAC'] = z_map['JAX']

if 'LVS' in favor_map:
    favor_map['LV'] = favor_map['LVS']
    z_map['LV'] = z_map['LVS']

sort_key = itemgetter(1)
sorted(favor_map.items(), key=sort_key)

print(spread_df)
z_scores

    Unnamed: 0 AwayTeam HomeTeam  PointSpread  OverUnder
0            0      LAR      SEA        -2.19      42.44
1            1      PHI      WAS         7.08      43.50
2            2       GB      CHI        -1.43      44.85
3            3       NE      BAL        -3.00      48.63
4            4      BUF      CLE        10.44      41.10
5            5      LAC      DAL        -1.11      50.52
6            6      NYJ       NO        -6.56      40.27
7            7       TB      CAR         2.98      44.60
8            8      MIN      NYG         2.50      41.48
9            9       KC      TEN         3.07      37.50
10          10      CIN      MIA         3.56      48.04
11          11      JAC      DEN        -3.43      46.72
12          12      ATL      ARI         2.68      47.68
13          13       LV      HOU       -14.24      39.44
14          14      PIT      DET        -7.58      51.98
15          15       SF      IND         5.56      46.00


{42.44: -0.5567192261592016,
 43.5: -0.292312671254131,
 44.85: 0.04443152603062827,
 48.63: 0.9873152784279536,
 41.1: -0.8909690219825911,
 50.52: 1.4587571546266163,
 40.27: -1.0980043432761832,
 44.6: -0.017928510503586348,
 41.48: -0.7961817664505859,
 37.5: -1.788953548075282,
 48.04: 0.8401455922072063,
 46.72: 0.5108845993065531,
 47.68: 0.7503471395979373,
 39.44: -1.305039664569777,
 51.98: 1.822939767986428,
 46.0: 0.33128769408801517}

In [134]:
# Debug: START OF FILTERING
print("\n=== START OF FILTERING CELL ===")
print(f"df shape at start: {df.shape}")
print(f"Positions in df: {df['Position'].unique()}")
print(f"Position counts at start:\n{df['Position'].value_counts()}")

set_teams = set(df['Team'])
# Automatically set SINGLE_GAME based on number of unique teams in the data
if len(set_teams) == 2:
    SINGLE_GAME = True
else:
    SINGLE_GAME = False

MIN_SALARY = 2000 if SINGLE_GAME else 4900
print('SINGLE_GAME', SINGLE_GAME, len(set_teams), set_teams, MIN_SALARY)
# df = df.fillna(df.median())
print(df.describe())

df['Name'] = df['First Name'] + " " + df['Last Name']
df['Salary/FPPG'] = df['FPPG'] / df['Salary']

# Players with injury status.
# for index, p in df[~df['Injury Indicator'].isna()].iterrows():
#     print(p['Injury Indicator'], p['Name'], p['Team'])

all_teams = favor_map.keys()
winning_teams = [x for x in all_teams if favor_map[x] < 0]
ordered_teams = sorted(all_teams, key=lambda x: favor_map[x])
print('ordered_teams', [(x, favor_map[x]) for x in ordered_teams])
# excluded_teams = set([x for x in all_teams if favor_map[x] > -MIN_FAVORED])
# top_teams = winning_teams
team_offset = 1
half_teams = int(len(ordered_teams)/2)
# For single-game contests, include all teams IN THE DATA. Otherwise, select top favored teams.
if SINGLE_GAME:
    top_teams = list(set_teams)  # Include only teams that are in the player data
else:
    top_teams = ordered_teams[team_offset:team_offset+int(half_teams)]  # Select middle to top (favored) teams
print(f"top_teams {top_teams}")
excluded_teams = set(all_teams) - set(top_teams)
print(f"excluded_teams {excluded_teams}")

questionable_players = list(df[(~df['Injury Indicator'].isna()) | (~df['Injury Details'].isna())]['Name'])
low_salary_players = list(df[((df['Salary'] < MIN_SALARY)) & (df['Position'] != 'D')]['Name'])

excluded_players = set([*questionable_players, *low_salary_players])

# Let the optimizer decide which defenses to use based on value/merit
# The adjusted projections already incorporate team quality via favor_map weighting

readd = ['Justin Herbert', 'Patrick Taylor Jr.', 'Bailey Zappe', "D'Andre Swift"]
for p in readd:
    if p in excluded_players:
        excluded_players.remove(p)

# excluded_players = ["Rob Gronkowski"]
print(f"Excluding: {len(excluded_players)}")

questionable_df = df[(df['Name'].isin(set(questionable_players)))]

excluded_df = df[(df['Name'].isin(excluded_players))]

df = df[(~df['Name'].isin(excluded_players))]# & (df['FPPG'] > 0) & (df['Played'] > 0)




=== START OF FILTERING CELL ===
df shape at start: (56, 18)
Positions in df: ['RB' 'QB' 'TE' 'WR' 'D' 'K']
Position counts at start:
Position
WR    20
RB    12
TE    11
QB     8
K      3
D      2
Name: count, dtype: int64
SINGLE_GAME True 2 {'SF', 'IND'} 2000
            FPPG     Played        Salary  MVP 1.5x Salary  Tier  Unnamed: 15  \
count  50.000000  56.000000     56.000000        56.000000   0.0          0.0   
mean    5.300982   7.000000   3878.571429      5817.857143   NaN          NaN   
std     6.083454   5.267395   3718.587918      5577.881877   NaN          NaN   
min    -0.100000   0.000000   1000.000000      1500.000000   NaN          NaN   
25%     0.426250   1.000000   1000.000000      1500.000000   NaN          NaN   
50%     2.501429   8.000000   1500.000000      2250.000000   NaN          NaN   
75%     8.100000  12.000000   6850.000000     10275.000000   NaN          NaN   
max    22.721429  15.000000  13000.000000     19500.000000   NaN          NaN   

       Un

In [135]:
# Debug: Check df after filtering
print("\n=== AFTER FILTERING ===")
print(f"df shape: {df.shape}")
print(f"Positions in df: {df['Position'].unique()}")
print(f"Position counts:\n{df['Position'].value_counts()}")
print(f"\nExcluded players: {len(excluded_players)}")
print(f"Sample excluded players: {list(excluded_players)[:10]}")



=== AFTER FILTERING ===
df shape: (25, 20)
Positions in df: ['RB' 'QB' 'TE' 'WR' 'D' 'K']
Position counts:
Position
WR    7
RB    6
QB    4
TE    4
D     2
K     2
Name: count, dtype: int64

Excluded players: 31
Sample excluded players: ['Spencer Shrader', 'Will Mallory', 'Anthony Gould', 'Ulysses Bentley IV', 'Jordan Watkins', 'Laquon Treadwell', 'Javon Baker', 'Ashton Dulin', 'Jordan James', 'Salvon Ahmed']


In [136]:
# print all in df
# for index, p in enumerate(low_salary_players):
#     print(p)


In [137]:
REPLACE_MAP = {
    'LA': 'Los Angeles',
    '.':'',
}

def name_map(x):
    result = ' '.join(x.split(', ')[::-1])
    for k in REPLACE_MAP:
        result = result.replace(k, REPLACE_MAP[k])
    return result

In [138]:
print(winning_teams)

['SEA', 'PHI', 'CHI', 'BAL', 'BUF', 'DAL', 'NO', 'TB', 'MIN', 'KC', 'CIN', 'DEN', 'ATL', 'HOU', 'DET', 'SF']


In [139]:
# http://rotoguru1.com/cgi-bin/fyday.pl?week=1&game=fd&scsv=1

start_week = WEEK-6# take last few games for momentum weighting.

file_names = [f"./history/week{week_number}.csv" for week_number in range(start_week, WEEK+1) if week_number != 18 and os.path.isfile(f"./history/week{week_number}.csv")]
print(file_names, len(file_names))

history_dfs = [pd.read_csv(f, delimiter=";") for f in file_names]
print(f"Using {len(history_dfs)} weeks of history")
historic_averages = {}
if history_dfs:
    historic_data=pd.concat(history_dfs)



    historic_data['Name'] = historic_data['Name'].apply(name_map)
    team_data = historic_data[historic_data['Pos'] == 'Def']

    historic_data[:1]
    historic_averages = historic_data.groupby("Name").mean()['FD points'].to_dict()
    historic_averages['Patrick Mahomes'] = historic_averages['Patrick Mahomes II']
    historic_averages['Darrell Henderson Jr'] = historic_averages['Darrell Henderson']
    # historic_averages

    team_averages = team_data.groupby("Team").mean()['FD points'].to_dict()
    team_averages['gb'] = team_averages.get('gnb')
    team_averages['kc'] = team_averages.get('kan')
    team_averages['ne'] = team_averages.get('nwe')
    team_averages['tb'] = team_averages.get('tam')
    team_averages['lv'] = team_averages.get('lvr')
    team_averages['no'] = team_averages.get('nor')
    team_averages['sf'] = team_averages.get('sfo')
    # print(team_averages)

    for k,v in team_averages.items():
        historic_averages[k] = team_averages[k]
    print(len(historic_averages))



[] 0
Using 0 weeks of history


In [140]:
excluded_bonus = defaultdict(lambda: 0)
injured_qb = defaultdict(lambda: False)

INJURY_FACTOR = .12

for index, p in questionable_df.iterrows():
    pos = p['Position']
    if pos in ['TE', 'WR', 'RB', 'QB']:
        points = p['FPPG']
        if points > 7.5 and p['Played'] >= WEEK / 2:
            injury_offset = min(points * INJURY_FACTOR, INJURY_FACTOR*10)
            if pos == 'QB':
                # subtract for QB
                amt = -injury_offset*2
                injured_qb[p['Team']] = True
            elif pos in ('RB', 'WR', 'TE'):
                amt = injury_offset*1.2
            else:
                amt = injury_offset

            print('bonus injured', p['Team'], p['Name'], amt)

            excluded_bonus[p['Team']] += amt

print(excluded_bonus)

bonus injured SF Ricky Pearsall 1.1807999725341796
bonus injured IND Daniel Jones -2.4
defaultdict(<function <lambda> at 0x16b07e660>, {'SF': 1.1807999725341796, 'IND': -2.4})


In [141]:
df.to_csv(ACTIVE_FILE)
print(f"wrote {ACTIVE_FILE}, {df.shape}")

wrote ./active/data.csv, (25, 20)


In [142]:
# https://www.fanduel.com/nfl-guide
# Use min/max + roster_size to accomodate flex position. FD roster_size = 9
all_positions = set(df['Position'])
print(all_positions)


def get_nfl_positions():
    if SINGLE_GAME:
        positions = [[p, 0, 5] for p in all_positions]
        positions.append(['MVP', 1, 1])  # Exactly 1 MVP required
        return positions

    return [
        ['QB', 1, 1],
        ['RB', 2, 3],
        ['WR', 3, 4],
        ['TE', 1, 2],
        ['D', 1, 1]
    ]

print(get_nfl_positions())

{'QB', 'TE', 'WR', 'RB', 'D', 'K'}
[['QB', 0, 5], ['TE', 0, 5], ['WR', 0, 5], ['RB', 0, 5], ['D', 0, 5], ['K', 0, 5], ['MVP', 1, 1]]


In [143]:
m_score = df.groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']

print(df.head())

# m_score
# print(df['Salary'])

             Id Position First Name             Nickname  Last Name  \
0  124667-55050       RB  Christian  Christian McCaffrey  McCaffrey   
1  124667-89675       RB   Jonathan      Jonathan Taylor     Taylor   
2  124667-85607       QB      Brock          Brock Purdy      Purdy   
3  124667-39793       TE     George        George Kittle     Kittle   
4   124667-7968       QB     Philip        Philip Rivers     Rivers   

        FPPG  Played  Salary  MVP 1.5x Salary    Game Team Opponent  \
0  21.942858      14   13000            19500  SF@IND   SF      IND   
1  22.721429      14   12600            18900  SF@IND  IND       SF   
2  19.063333       6   11800            17700  SF@IND   SF      IND   
3  11.844444       9   11200            16800  SF@IND   SF      IND   
4   7.800000       1   10200            15300  SF@IND  IND       SF   

  Injury Indicator Injury Details  Tier  Unnamed: 15  Unnamed: 16  \
0              NaN            NaN   NaN          NaN          NaN   
1       

In [144]:
# Debug: Check what's in df before groupby
print("DataFrame shape:", df.shape)
print("Positions in df:", df['Position'].unique())
print("Number of each position:", df['Position'].value_counts())
print("\nFirst few rows:")


DataFrame shape: (25, 20)
Positions in df: ['RB' 'QB' 'TE' 'WR' 'D' 'K']
Number of each position: Position
WR    7
RB    6
QB    4
TE    4
D     2
K     2
Name: count, dtype: int64

First few rows:


In [145]:
max_salary = np.percentile(df[df['Salary'] >= MIN_SALARY]['Salary'], 99)
max_salary = 9400
print(max_salary)

9400


In [146]:
qbs = df[df['Position'] == 'QB']
print(qbs[:10])

               Id Position First Name       Nickname Last Name       FPPG  \
2    124667-85607       QB      Brock    Brock Purdy     Purdy  19.063333   
4     124667-7968       QB     Philip  Philip Rivers    Rivers   7.800000   
9   124667-154450       QB      Riley  Riley Leonard   Leonard   5.700000   
10   124667-90584       QB        Mac      Mac Jones     Jones  14.534000   

    Played  Salary  MVP 1.5x Salary    Game Team Opponent Injury Indicator  \
2        6   11800            17700  SF@IND   SF      IND              NaN   
4        1   10200            15300  SF@IND  IND       SF              NaN   
9        2    8000            12000  SF@IND  IND       SF              NaN   
10      10    8000            12000  SF@IND   SF      IND              NaN   

   Injury Details  Tier  Unnamed: 15  Unnamed: 16            Roster Position  \
2             NaN   NaN          NaN          NaN  MVP - 1.5X Points/AnyFLEX   
4             NaN   NaN          NaN          NaN  MVP - 1.5X P

In [147]:
ACTIVE_RULE_SET = rules.FD_NFL_RULE_SET
# Overrides (position limits, salary, roster size, positions, etc.
ACTIVE_RULE_SET.salary_max = 60000

ACTIVE_RULE_SET.defensive_positions = ['D', 'DEF']
ACTIVE_RULE_SET.offensive_positions = ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX', 'K', 'MVP'] if SINGLE_GAME else ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX', 'K']
ACTIVE_RULE_SET.position_limits = get_nfl_positions()
ACTIVE_RULE_SET.salary_min = ACTIVE_RULE_SET.salary_max - (200 if SINGLE_GAME else 100)

if not SINGLE_GAME:
  ACTIVE_RULE_SET.max_players_per_team = 9

ACTIVE_RULE_SET.roster_size = 9 if not SINGLE_GAME else 6
print(ACTIVE_RULE_SET.__dict__)

ALL_POSITIONS = [*ACTIVE_RULE_SET.defensive_positions, *ACTIVE_RULE_SET.offensive_positions]

{'site': 'FAN_DUEL', 'league': 'NFL', 'roster_size': 6, 'position_limits': [['QB', 0, 5], ['TE', 0, 5], ['WR', 0, 5], ['RB', 0, 5], ['D', 0, 5], ['K', 0, 5], ['MVP', 1, 1]], 'general_position_limits': [], 'salary_min': 59800, 'salary_max': 60000, 'offensive_positions': ['QB', 'RB', 'WR', 'TE', 'FLEX', 'WR/FLEX', 'K', 'MVP'], 'defensive_positions': ['D', 'DEF'], 'game_type': 'classic', 'max_players_per_team': 8, 'position_per_team_rules': None, 'min_teams': None, 'min_matchups': None, 'custom_rules': None}


In [148]:
# Any additional player or custom rule constraints.
#Player -  https://github.com/BenBrostoff/draftfast/blob/68625902ceea83e66ee9f13a44acd732f600f68f/draftfast/orm.py#L245

# no players min cost (unlikely to play), low score, or favored to lose except overwhelming proj.
# Use salary data from csv as optimization basis.
# set((k, v['defensive_rating_rank']) for k,v in rankings.items())
# rankings['BUF']

allowed_map = get_fantasy_def_points_against(WEEK)

if 'Washington Football Team' in allowed_map:
    allowed_map['Washington Commanders'] = allowed_map['Washington Football Team']
allowed_map

return cached data week 16


{}

In [149]:
print(z_map)

{'SEA': -0.5567192261592016, 'LAR': -0.5567192261592016, 'WAS': -0.292312671254131, 'PHI': -0.292312671254131, 'CHI': 0.04443152603062827, 'GB': 0.04443152603062827, 'BAL': 0.9873152784279536, 'NE': 0.9873152784279536, 'CLE': -0.8909690219825911, 'BUF': -0.8909690219825911, 'DAL': 1.4587571546266163, 'LAC': 1.4587571546266163, 'NO': -1.0980043432761832, 'NYJ': -1.0980043432761832, 'CAR': -0.017928510503586348, 'TB': -0.017928510503586348, 'NYG': -0.7961817664505859, 'MIN': -0.7961817664505859, 'TEN': -1.788953548075282, 'KC': -1.788953548075282, 'MIA': 0.8401455922072063, 'CIN': 0.8401455922072063, 'DEN': 0.5108845993065531, 'JAC': 0.5108845993065531, 'ARI': 0.7503471395979373, 'ATL': 0.7503471395979373, 'HOU': -1.305039664569777, 'LV': -1.305039664569777, 'DET': 1.822939767986428, 'PIT': 1.822939767986428, 'IND': 0.33128769408801517, 'SF': 0.33128769408801517}


In [150]:
players = salary_download.generate_players_from_csvs(salary_file_location=ACTIVE_FILE, game=rules.FAN_DUEL)

FAVOR_DIVISION = 4
AVERAGE_WEIGHT = .5

defenses = []
qbs = []
mvps = []  # Track MVP players separately
mvp_base_names = set()  # Track base names of MVP players

MIN_QB_SALARY = 1000 if SINGLE_GAME else 6400

MIN_SCORE = 7
MAX_SCORE = 27
INJURED_QB_BONUS = 1.25
HOME_BONUS = .3

historic_data_used = 0

def build_starter_map(players, questionable_df, MIN_PLAYED):
    """
    Build a map of (team, position) -> (starter_name, starter_fppg, is_injured).
    
    Only tracks positions: QB, RB, WR, TE
    Identifies starters as the highest FPPG player for each position/team combo.
    Checks if starters are injured by looking them up in questionable_df.
    
    Args:
        players: List of Player objects
        questionable_df: DataFrame of players with injury indicators
        MIN_PLAYED: Minimum games played to be considered a starter
    
    Returns:
        dict: Mapping (team, pos) -> (starter_name, starter_fppg, is_injured)
    """
    starter_map = {}
    for p in players:
        if p.pos in ['QB', 'RB', 'WR', 'TE']:
            key = (p.team, p.pos)
            base_name = p.name.replace(' (MVP)', '')
            current_fppg = float(p.kv_store.get('FPPG', 0))
            
            # Check if this player is injured
            is_injured = base_name in [q['Name'] for _, q in questionable_df.iterrows()] if not questionable_df.empty else False
            
            # Only store if this is the highest FPPG for this position/team (starter)
            if key not in starter_map or current_fppg > starter_map[key][1]:
                starter_map[key] = (base_name, current_fppg, is_injured)
    
    return starter_map


def filter_mvps(mvps, players, starter_map, MIN_PLAYED):
    """
    Filter MVP players based on games played and starter injury status.
    
    Includes a player if:
    - Player has played >= MIN_PLAYED games (is a starter), OR
    - Player is a backup AND their position's starter is injured
    
    Args:
        mvps: List of tuples (name, proj, cost, value, pos, base_fppg)
        players: List of Player objects (to look up games played and team info)
        starter_map: Dict mapping (team, pos) -> (starter_name, starter_fppg, is_injured)
        MIN_PLAYED: Minimum games played threshold for starters
    
    Returns:
        list: Filtered MVP tuples with added fields (player_games, status)
              Format: (name, proj, cost, value, pos, base_fppg, player_games, status)
    """
    filtered_mvps = []
    
    for name, proj, cost, value, pos, base_fppg in sorted(mvps, key=lambda x: x[3], reverse=True):
        # Extract the base player name (remove " (MVP)" suffix)
        base_name = name.replace(' (MVP)', '')
        
        # Find the player in the original players list to check games played and team
        player_games = 0
        player_team = None
        player_pos = None
        for p in players:
            if p.name == base_name:
                player_games = int(float(p.kv_store.get('Played', 0)))
                player_team = p.team
                player_pos = p.pos
                break
        
        status = "Starter"
        is_backup = player_games < MIN_PLAYED
        
        # Check if this is a backup but their starter is injured
        starter_injured = False
        if is_backup and player_team and player_pos in ['QB', 'RB', 'WR', 'TE']:
            key = (player_team, player_pos)
            if key in starter_map:
                starter_name, starter_fppg, is_injured = starter_map[key]
                if is_injured:
                    starter_injured = True
                    status = f"(Starter {starter_name} injured)"
        
        # Include if starter, or if backup but starter is injured
        if player_games >= MIN_PLAYED or starter_injured:
            filtered_mvps.append((name, proj, cost, value, pos, base_fppg, player_games, status))
    
    return filtered_mvps

def calculate_home_bonus(p):
    """Calculate home/away bonus."""
    teams = p.matchup.split('@')
    is_home = p.team == teams[1]
    return HOME_BONUS if is_home else -HOME_BONUS

def calculate_overunder_bonus(p, point_bonus):
    """Calculate over/under bonus based on position.
    For defenses, heavily weight the over/under since they score better in low-scoring games.
    """
    if not point_bonus:
        return 0
    if p.pos in ['D', 'MVP']:
        # For defenses: heavily penalize high O/U games, heavily reward low O/U games
        # Multiply by larger factor (3x vs 1.5x for other positions)
        return -point_bonus * 3.0
    return point_bonus * 1.5

def calculate_ranking_bonus(p, opponent):
    """Calculate ranking-based bonus."""
    current_rank = rankings.get(p.team, {}).get('points_rank_def', 0) if 'rankings' in globals() else 0
    opp_rank = (num_teams - rankings.get(opponent, {}).get('offensive_yards_rank', 0)) if 'rankings' in globals() else 0
    overall_diff = opp_rank - current_rank
    return overall_diff / num_teams

def calculate_injury_bonuses(p, opponent):
    """Calculate all injury-related bonuses."""
    bonuses = 0

    # Opponent QB injury bonus
    if injured_qb.get(opponent, False):
        qb_bonus = INJURED_QB_BONUS if p.pos in ['D', 'MVP'] else INJURED_QB_BONUS / 2
        # print(f"hurt {opponent} QB, {qb_bonus} bonus to {p.pos} {p.team} {p.name}")
        bonuses += qb_bonus

    # Team injury bonus
    if p.pos == 'QB':
        bonuses += -excluded_bonus.get(p.team, 0)
    elif p.pos in ['D', 'MVP']:
        bonuses += excluded_bonus.get(p.team, 0) / 2
    else:
        bonuses += excluded_bonus.get(p.team, 0)

    # RB bonus for injured QB on own team
    if p.pos == 'RB' and injured_qb.get(p.team, False):
        # print(f"hurt {p.team} QB, {INJURED_QB_BONUS} bonus to {p.pos} {p.name}")
        bonuses += INJURED_QB_BONUS

    return bonuses

def get_blended_projection(p, history_key):
    """
    Helper function to blend current projection with historical average.
    Returns blended projection or original projection if no history available.
    Increments historic_data_used counter when blending occurs.
    """
    global historic_data_used
    history_value = historic_averages.get(history_key)
    if history_value:
        historic_data_used += 1
        return AVERAGE_WEIGHT * p.proj + (1 - AVERAGE_WEIGHT) * history_value
    return p.proj

def calculate_adjusted_projection(p):
    """
    Calculate the adjusted projection for a player based on matchup factors.
    
    Base score is blended with historical averages, then adjusted by:
    - Spread advantage: How favorable the matchup is (Vegas spread)
    - Game script: Over/under to identify high/low scoring games
    - Home field advantage: Home vs away boost/penalty
    - Opponent strength: How good opponent defense is against this position
    - Team health: Injuries affecting the team or position
    
    Kickers are excluded from matchup weighting and use simple historical average blending.
    
    Returns a projection capped between MIN_SCORE and MAX_SCORE.
    """
    if not WEIGHTED:
        return p.proj

    # Kickers skip detailed weighting - just blend with historical average
    if p.pos == 'K':
        return get_blended_projection(p, name_map(p.name))

    # Low-salary players skip detailed weighting - just blend with historical average
    if p.cost <= 4200:
        history_key = name_map(p.name) if p.pos not in ['D', 'MVP'] else p.team.lower()
        return get_blended_projection(p, history_key)

    # Step 1: Blend current projection with historical average
    # Historical data provides longer-term expectations
    history_key = name_map(p.name) if p.pos not in ['D', 'MVP'] else p.team.lower()
    base_score = get_blended_projection(p, history_key)

    # Step 2: Identify opponent for this matchup
    teams = p.matchup.split('@')
    opponent = teams[0] if p.team == teams[1] else teams[1]

    # Step 3: Calculate matchup adjustments
    matchup_bonus = 0

    # GAME TOTAL ADJUSTMENT: Over/under indicates high/low scoring game
    # Calculate this first since it affects spread weighting
    # High O/U = more total points in game = higher individual scoring
    # For defenses, high O/U is bad (more opponent points)
    point_bonus = z_map.get(p.team, 0)
    ou_bonus = calculate_overunder_bonus(p, point_bonus)
    matchup_bonus += ou_bonus

    # SPREAD ADJUSTMENT: Vegas spread indicates how favorable the matchup is
    # For OFFENSIVE players: positive spread = unfavored = fewer scoring opportunities
    # For DEFENSES: positive spread = facing strong offense = worse matchup
    spread_bonus = -favor_map.get(p.team, 0) / FAVOR_DIVISION
    
    # For defenses, invert the spread bonus (they benefit from facing bad offenses)
    if p.pos in ['D', 'MVP']:
        spread_bonus = -favor_map.get(p.team, 0) / FAVOR_DIVISION
        # In high-scoring games, even strong defenses give up points
        # So heavily dampen spread advantage in high O/U games
        spread_weight = 1.0 - (point_bonus * 0.5)  # Stronger dampening for defenses
    else:
        # In high-scoring games, spread matters less (everyone scores more)
        # In low-scoring games, spread matters more (must be on favored side)
        spread_weight = 1.0 - (point_bonus * 0.4)  # Moderate dampening for offensive players
    
    spread_bonus *= spread_weight
    matchup_bonus += spread_bonus

    # HOME FIELD ADVANTAGE: Home teams score more, away teams score less
    # This applies to all positions
    home_bonus = calculate_home_bonus(p)
    matchup_bonus += home_bonus

    # OPPONENT STRENGTH: How good is opponent defense?
    # (Uses rankings if available)
    ranking_bonus = calculate_ranking_bonus(p, opponent)
    matchup_bonus += ranking_bonus

    # TEAM HEALTH ADJUSTMENTS: How are injuries affecting this team/position?
    injury_bonus = calculate_injury_bonuses(p, opponent)
    matchup_bonus += injury_bonus

    # DEFENSIVE MATCHUP: For defenses and MVPs, account for opponent offense strength
    if p.pos in ['D', 'MVP']:
        matchup_bonus += excluded_bonus.get(opponent, 0) / 4

    # Step 4: Apply adjustments with safeguards
    # Only apply if player has minimum baseline or is a defense/MVP
    if p.pos in ['D', 'MVP'] or base_score >= MIN_SCORE:
        adjusted_proj = min(base_score + matchup_bonus, MAX_SCORE)
        return adjusted_proj

    return base_score

# First, adjust all regular players
for p in players:
    p.average_score = m_score[p.pos if p.pos in m_score else p.pos.replace('MVP','D')]
    name = p.name.replace('.', '')

    # Calculate adjusted projection for regular players
    original_proj = p.proj
    p.proj = calculate_adjusted_projection(p)

    # Store the adjusted projection for MVP creation
    p.kv_store['adjusted_proj'] = p.proj

# For single game, create MVP slot with 1.5x salary and 1.5x ADJUSTED projection
if SINGLE_GAME:
    mvp_players = []
    regular_players = []

    for p in players:
        import copy
        mvp = copy.deepcopy(p)
        mvp.pos = 'MVP'
        mvp.cost = int(round(p.cost * 1.5))
        # Use the adjusted projection for the base, then apply MVP boost
        adjusted_base = p.kv_store.get('adjusted_proj', p.proj)
        mvp.proj = adjusted_base * 1.5
        mvp.name = p.name + ' (MVP)'
        mvp.kv_store['base_name'] = p.name  # Store original name
        mvp_players.append(mvp)
        mvp_base_names.add(p.name)

        # Keep regular players but mark them for potential exclusion
        regular_players.append(p)

    # For single game, we want EITHER the regular OR MVP version, not both
    # We'll handle this in the optimization constraints
    players.extend(mvp_players)

# Validate that all player teams are in spread data
teams_in_data = set(favor_map.keys())
missing_teams = set()

for p in players:
    if p.team not in teams_in_data:
        missing_teams.add(p.team)

if missing_teams:
    error_msg = "The following teams are missing from spread/matchup data:\n"
    for team in sorted(missing_teams):
        player_count = sum(1 for p in players if p.team == team)
        error_msg += f"  {team} ({player_count} players affected)\n"
    raise ValueError(error_msg)

# Now sort players into appropriate lists (including MVPs)
for p in players:
    # Sort players into appropriate lists
    base_fppg = float(p.kv_store.get('FPPG', 0))
    if p.pos == 'MVP':
        mvps.append((p.name, p.proj, p.cost, p.proj / p.cost, p.pos, base_fppg))
    elif p.pos == 'D':
        spread = favor_map.get(p.team, 0)
        defenses.append((p.team, p.proj, p.cost, p.proj / p.cost, base_fppg, spread))
    elif p.pos == 'QB' and p.cost >= MIN_QB_SALARY:
        # print('QB', p.name, p.cost, p.proj)
        point_bonus = z_map.get(p.team, 0)
        favor_bonus = -favor_map.get(p.team, 0) / FAVOR_DIVISION
        base_fppg = float(p.kv_store.get('FPPG', 0))
        qbs.append((name_map(p.name), p.proj, p.cost, p.proj / p.cost, point_bonus, favor_bonus, base_fppg))

print(f"players {len(players)}")
print(f"historic data used {historic_data_used} of {len(players)}")

# Define MIN_PLAYED here for use in MVP filtering below
MIN_PLAYED = max(int(WEEK * 0.6), 8)  # At least 60% of season or 8 games minimum

# Build the starter map and filter MVPs using extracted functions
starter_map = build_starter_map(players, questionable_df, MIN_PLAYED)
filtered_mvps = filter_mvps(mvps, players, starter_map, MIN_PLAYED)


players 50
historic data used 0 of 50


In [151]:

# Best picks
# Define MIN_PLAYED here for use in MVP filtering below
MIN_PLAYED = max(int(WEEK * 0.6), 8)  # At least 60% of season or 8 games minimum

print("\n" + "="*100)
print("SORTED DEFENSES")
print("="*100)
print(f"{'Team':<8} {'Proj':>8} {'Salary':>10} {'Base':>8} {'Spread':>8} {'Value':>8}")
print("-"*100)
for team, proj, cost, value, base_fppg, spread in sorted(defenses, key=lambda x: x[3], reverse=True):
    print(f"{team:<8} {proj:>8.2f} ${cost:>9,.0f} {base_fppg:>8.2f} {spread:>8.2f} {(value*1000):>7.1f}x")

if SINGLE_GAME:
    print("\n" + "="*100)
    print("SORTED MVPs (Top 10 - Backups Shown Only if Starter Injured)")
    print("="*100)
    print(f"{'Name':<35} {'Proj':>8} {'Salary':>10} {'Base':>8} {'Value':>8} {'Games':>7} {'Status':<10}")
    print("-"*100)
    
    # filtered_mvps and starter_map are already computed from the previous cell
    # Display the filtered MVPs
    for name, proj, cost, value, pos, base_fppg, games_played, status in filtered_mvps[:10]:
        print(f"{name:<35} {proj:>8.2f} ${cost:>9,.0f} {base_fppg:>8.2f} {(value*1000):>7.1f}x {games_played:>7.0f} {status:<10}")



SORTED DEFENSES
Team         Proj     Salary     Base   Spread    Value
----------------------------------------------------------------------------------------------------
SF           6.53 $    7,000     4.43    -5.56     0.9x
IND          5.03 $    6,000     6.79     5.56     0.8x

SORTED MVPs (Top 10 - Backups Shown Only if Starter Injured)
Name                                    Proj     Salary     Base    Value   Games Status    
----------------------------------------------------------------------------------------------------
Mac Jones (MVP)                        24.57 $   12,000    14.53     2.0x      10 Starter   
Christian McCaffrey (MVP)              39.23 $   19,500    21.94     2.0x      14 Starter   
Jonathan Taylor (MVP)                  33.24 $   18,900    22.72     1.8x      14 Starter   
Eddy Pineiro (MVP)                     16.09 $   10,200    10.73     1.6x      11 Starter   
Jauan Jennings (MVP)                   20.50 $   14,100     9.46     1.5x      12 Star

In [152]:

print("\n" + "="*105)
print("SORTED QBs")
print("="*105)
print(f"{'Name':<35} {'Proj':>8} {'Salary':>10} {'Value':>8} {'O/U':>8} {'Spread':>8} {'Base':>8}")
print("-"*105)
for name, proj, cost, value, ou_bonus, spread_bonus, base_fppg in sorted(qbs, key=lambda x: x[-1], reverse=True):
    print(f"{name:<35} {proj:>8.2f} ${cost:>9,.0f} {(value*1000):>7.1f}x {ou_bonus:>8.2f} {spread_bonus:>8.2f} {base_fppg:>8.2f}")


SORTED QBs
Name                                    Proj     Salary    Value      O/U   Spread     Base
---------------------------------------------------------------------------------------------------------
Brock Purdy                            20.91 $   11,800     1.8x     0.33     1.39    19.06
Mac Jones                              16.38 $    8,000     2.0x     0.33     1.39    14.53
Philip Rivers                          10.79 $   10,200     1.1x     0.33    -1.39     7.80
Riley Leonard                           5.70 $    8,000     0.7x     0.33    -1.39     5.70


In [153]:
# resets
LOCKED = []
BANNED = []
BLOCKED_TEAMS = []

player_settings = PlayerPoolSettings()

MIN_PROJ = 0
# For single-game contests, require players to have started in at least 60% of games (excludes backups)
MIN_PLAYED = max(int(WEEK * 0.6), 8)  # At least 60% of season or 8 games minimum

# Positive value means allow teams that are unfavored to win.
min_favored = 10
MIN_LIMIT = min_favored - 2

roster = None
best_roster = None
best_score = 0

get_score = lambda roster: sum([p.proj for p in roster.players])

def block_function(p):
    store = p.kv_store
    played = int(float(store.get('Played') or 0))
    if SINGLE_GAME:
        # require min number of games played
        return played < MIN_PLAYED
        # return False
    name = p.name if p.pos != 'D' else p.team

    if p.team in BLOCKED_TEAMS:
        return True

    if p.pos == 'D' and (p.cost > 5000):
        return True

    if p.pos == 'QB' and p.cost < MIN_QB_SALARY or p.cost > max_salary:
        return True

    # print(favor_map[name], p.__dict__, min_favored)
    cost_filter = p.pos != 'QB' and (p.cost > max_salary or played <1)# or played > WEEK+1)

    should_skip = (p.proj < MIN_PROJ and p.pos != 'D') or (p.proj < 10 and p.pos == 'QB')  or cost_filter

    #print(p.name, played, MAX_PLAYED)
    return should_skip

def print_optimized_roster(roster, min_favored_factor, iteration_num):
    """
    Print an optimized roster in table format with weighting factor info.
    Sorted by position order: QB, RBs, WRs, FLEX, TE, D
    
    Args:
        roster: The optimized NFL roster
        min_favored_factor: The current min_favored weighting factor
        iteration_num: Which iteration of the optimization loop this is
    """
    print("\n" + "="*120)
    print(f"OPTIMIZED LINEUP #{iteration_num} (weighting_factor={min_favored_factor}, total_score={get_score(roster):.2f})\n---")
    
    # Define position order
    position_order = {'QB': 0, 'RB': 1, 'WR': 2, 'TE': 3, 'D': 4, 'MVP': 5, 'FLEX': 6}
    
    # Create table data and sort by position
    roster_data = []
    total_salary = 0
    for i, p in enumerate(roster.players, 1):
        # Get base projection (original FPPG) and adjusted projection
        base_fppg = float(p.kv_store.get('FPPG', 0))
        adjusted_proj = p.proj
        salary = int(p.cost)  # Convert to int to avoid decimal point formatting issues
        
        # Extract opponent from matchup (format: "AwayTeam@HomeTeam")
        if p.matchup:
            teams = p.matchup.split('@')
            if len(teams) == 2:
                opponent = teams[0].strip() if p.team == teams[1].strip() else teams[1].strip()
            else:
                opponent = 'N/A'
        else:
            opponent = 'N/A'
        
        # Get spread from favor_map (already properly signed: + for favorites, - for underdogs)
        spread = favor_map.get(p.team, 0)
        
        total_salary += salary
        
        # Get position order for sorting (use the mapped value, default to 99 if not found)
        pos_order = position_order.get(p.pos, 99)
        
        roster_data.append({
            'Slot': f"{p.pos:5}",
            'Name': p.name[:25],
            'Team': p.team,
            'Opp': opponent,
            'Salary': f"${salary:,}",
            'Base Proj': f"{base_fppg:6.2f}",
            'Weighted Proj': f"{adjusted_proj:6.2f}",
            'Value': f"{(adjusted_proj/salary)*1000:6.1f}x",
            'Spread': f"{spread:+.1f}",
            'pos_order': pos_order
        })
    
    # Sort by position order
    roster_data.sort(key=lambda x: x['pos_order'])
    
    # Print table header
    print(f"{'Slot':<6} {'Name':<27} {'Team':<5} {'Opp':<5} {'Salary':<10} {'Base Proj':<12} {'Weighted Proj':<15} {'Value':<10} {'Spread':<8}")
    print("-"*120)
    
    # Print rows
    for row in roster_data:
        print(f"{row['Slot']:<6} {row['Name']:<27} {row['Team']:<5} {row['Opp']:<5} {row['Salary']:<10} {row['Base Proj']:<12} {row['Weighted Proj']:<15} {row['Value']:<10} {row['Spread']:<8}")
    
    # Print summary
    print("-"*120)
    total_salary_formatted = f"{total_salary:,}"
    print(f"{'TOTAL':<6} {'':<27} {'':<5} ${total_salary_formatted:<9} {'':<12} {get_score(roster):<15.2f}")
    print("-"*3)


In [154]:

def build_optimizer_settings(players, block_function):
    """
    Build optimizer settings with custom rules to enforce position constraints.
    
    Rules created:
    - Block players according to block_function
    - At most 1 RB per team
    - At most 1 WR per team
    
    Args:
        players: List of Player objects
        block_function: Function to determine which players to exclude
    
    Returns:
        OptimizerSettings object configured with all custom rules
    """
    custom_rules = [
        CustomRule(
            group_a=lambda p: p,
            group_b=block_function,
            comparison=lambda sum, a, b: sum(b) == 0
        )
    ]

    # Group RBs by team and enforce max 1 per team
    rb_teams = {}
    for p in players:
        if p.pos == 'RB':
            if p.team not in rb_teams:
                rb_teams[p.team] = []
            rb_teams[p.team].append(p)

    # Add constraint: at most 1 RB per team
    for team, team_rbs in rb_teams.items():
        if len(team_rbs) > 1:
            custom_rules.append(
                CustomRule(
                    group_a=lambda p, team=team: p.pos == 'RB' and p.team == team,
                    group_b=lambda p: False,
                    comparison=lambda sum, a, b: sum(a) <= 1
                )
            )

    # Group WRs by team and enforce max 1 per team
    wr_teams = {}
    for p in players:
        if p.pos == 'WR':
            if p.team not in wr_teams:
                wr_teams[p.team] = []
            wr_teams[p.team].append(p)

    # Add constraint: at most 1 WR per team
    for team, team_wrs in wr_teams.items():
        if len(team_wrs) > 1:
            custom_rules.append(
                CustomRule(
                    group_a=lambda p, team=team: p.pos == 'WR' and p.team == team,
                    group_b=lambda p: False,
                    comparison=lambda sum, a, b: sum(a) <= 1
                )
            )

    # Group QBs by team and enforce max 1 per team
    qb_teams = {}
    for p in players:
        if p.pos == 'QB':
            if p.team not in qb_teams:
                qb_teams[p.team] = []
            qb_teams[p.team].append(p)

    # Add constraint: at most 1 QB per team
    for team, team_qbs in qb_teams.items():
        if len(team_qbs) > 1:
            custom_rules.append(
                CustomRule(
                    group_a=lambda p, team=team: p.pos == 'QB' and p.team == team,
                    group_b=lambda p: False,
                    comparison=lambda sum, a, b: sum(a) <= 1
                )
            )

    # Group TEs by team and enforce max 1 per team
    te_teams = {}
    for p in players:
        if p.pos == 'TE':
            if p.team not in te_teams:
                te_teams[p.team] = []
            te_teams[p.team].append(p)

    # Add constraint: at most 1 TE per team
    for team, team_tes in te_teams.items():
        if len(team_tes) > 1:
            custom_rules.append(
                CustomRule(
                    group_a=lambda p, team=team: p.pos == 'TE' and p.team == team,
                    group_b=lambda p: False,
                    comparison=lambda sum, a, b: sum(a) <= 1
                )
            )

    return OptimizerSettings(
        custom_rules=custom_rules,
        min_teams=3 if not SINGLE_GAME else 2
    )

In [155]:
BLOCKED_TEAMS = []

if SINGLE_GAME:
    LOCKED = ['Lamar Jackson']
    BANNED = ['Tyler Huntley',  'Kyle Williams', 'Patrick Taylor Jr.', 'Mac Jones', 'Mac Jones (MVP)', 'Philip Rivers']
else:
    LOCKED = ['Jared Goff', 'Denver Broncos', 'RJ Harvey']
    BANNED = ['Washington Commanders', 'Jacory Croskey-Merritt', 'Jaxon Smith-Njigba', 'JuJu Smith-Schuster', 
              'Minnesota Vikings', 'George Pickens', 'Jordan Addison', 'KaVontae Turpin', 'Evan Engram', 
              'Xavier Worthy', 'James Cook III', 'Jalen Nailor', 'T.J. Hockenson']

constraints = LineupConstraints(locked=LOCKED, banned=BANNED)

# Keep optimizing until we get a valid roster without duplicate base players
valid_roster_found = False
max_iterations = 10
iteration_count = 0


In [156]:

while not valid_roster_found and iteration_count < max_iterations:
    iteration_count += 1
    
    # Build optimizer settings with custom rules
    opt_settings = build_optimizer_settings(players, block_function)

    roster = run(
        rule_set=ACTIVE_RULE_SET,
        player_pool=players,
        verbose=False,
        optimizer_settings=opt_settings,
        constraints=constraints,
        player_settings=player_settings
    )

    if roster:
        # Check for duplicate base players in single game mode
        if SINGLE_GAME:
            # Group players by base name
            base_name_groups = {}
            for p in roster.players:
                base_name = p.name.replace(' (MVP)', '')
                if base_name not in base_name_groups:
                    base_name_groups[base_name] = []
                base_name_groups[base_name].append(p)

            # Find groups with duplicates and decide which to ban
            players_to_ban = []
            for base_name, player_group in base_name_groups.items():
                if len(player_group) > 1:
                    # We have duplicates - prefer MVP version if it exists
                    mvp_players = [p for p in player_group if ' (MVP)' in p.name]
                    regular_players = [p for p in player_group if ' (MVP)' not in p.name]

                    # Check if any player in this group is locked
                    locked_players = [p for p in player_group if p.name in LOCKED]

                    if locked_players:
                        # If a player is locked, ban all others in the group
                        for p in player_group:
                            if p.name not in LOCKED:
                                players_to_ban.append(p.name)
                                print(f"Banning {p.name} (keeping locked version {[lp.name for lp in locked_players]})")
                    elif mvp_players:
                        # Keep the MVP, ban the regular version(s)
                        for regular_p in regular_players:
                            players_to_ban.append(regular_p.name)
                            print(f"Banning regular player {regular_p.name} (keeping MVP version)")
                    else:
                        # No MVP version, keep the first regular player, ban others
                        for i, regular_p in enumerate(regular_players):
                            if i > 0:
                                players_to_ban.append(regular_p.name)
                                print(f"Banning duplicate player {regular_p.name}")

            if players_to_ban:
                print(f"Found duplicate base players, banning: {players_to_ban}")
                BANNED.extend(players_to_ban)
                # Remove any locked players from BANNED before updating constraints
                BANNED = [p for p in BANNED if p not in LOCKED]
                constraints = LineupConstraints(locked=LOCKED, banned=BANNED)
                continue  # Re-optimize with updated ban list
            
            # No duplicates found, roster is valid
            valid_roster_found = True
        else:
            # Not single game, roster is valid
            valid_roster_found = True
    
    if valid_roster_found and roster:
        # Print optimized roster
        print_optimized_roster(roster, min_favored, iteration_count)
        
        current_score = get_score(roster)
        if not best_score or current_score > best_score:
            best_score = current_score
            best_roster = roster

if not roster:
    print("No solution")
elif not favor_map:
    print("Warning: No favor map used")

Banning regular player Christian McCaffrey (keeping MVP version)
Found duplicate base players, banning: ['Christian McCaffrey']

OPTIMIZED LINEUP #2 (weighting_factor=10, total_score=97.13)
---
Slot   Name                        Team  Opp   Salary     Base Proj    Weighted Proj   Value      Spread  
------------------------------------------------------------------------------------------------------------------------
RB     Jonathan Taylor             IND   SF    $12,600     22.72        22.16             1.8x    +5.6    
RB     Brian Robinson Jr.          SF    IND   $5,400       3.74         3.74             0.7x    -5.6    
WR     Jauan Jennings              SF    IND   $9,400       9.46        13.67             1.5x    -5.6    
MVP    Christian McCaffrey (MVP)   SF    IND   $19,500     21.94        39.23             2.0x    -5.6    
K      Eddy Pineiro                SF    IND   $6,800      10.73        10.73             1.6x    -5.6    
K      Blake Grupe                 IND   SF

In [157]:
# def find_id(first_name, last_name, pos):
#     matches = df.loc[(df['First Name'] == first_name) & (df['Last Name'] == last_name) & (df['Position'] == pos)]
#     if matches:||
#         return matches.iloc[0]['Id']
#     return None
#print(f"{"Player":20}{"Advantage":10}")
if best_roster:
    roster = best_roster
    sorted_players = sorted(roster.players, key=lambda x: favor_map[x.team])
    net_score = 0
    for p in sorted_players:
        advantage = favor_map[p.team]
        name = p.name.replace('.', '')
        print(f"{name:24}{advantage:>10}{historic_averages.get(name, ''):>20} {p.proj}  {excluded_bonus[p.team]}")#, p.kv_store['Played'], MAX_PLAYED)
        net_score += advantage
    print(f"---\nTotal adv: {net_score}\n")
    roster.players[-1].__dict__



Jauan Jennings               -5.56                     13.6668688890866  1.1807999725341796
Eddy Pineiro                 -5.56                     10.727272727272728  1.1807999725341796
Brian Robinson Jr            -5.56                     3.7357142312186102  1.1807999725341796
Christian McCaffrey (MVP)     -5.56                     39.22709035581182  1.1807999725341796
Jonathan Taylor               5.56                     22.162556506438932  -2.4
Blake Grupe                   5.56                     7.615384615384615  -2.4
---
Total adv: -11.120000000000001



In [158]:
for r in roster.players:
    print(r.team, r.name, r.proj, r.average_score, r.cost,  r.kv_store['Injury Indicator'], r.kv_store['Injury Details'], r.kv_store['Played'])

IND Jonathan Taylor 22.162556506438932 9.582500189826602 12600.0   14
SF Jauan Jennings 13.6668688890866 6.645921938256358 9400.0   12
SF Eddy Pineiro 10.727272727272728 9.171328671328672 6800.0   11
IND Blake Grupe 7.615384615384615 9.171328671328672 6200.0   13
SF Brian Robinson Jr. 3.7357142312186102 9.582500189826602 5400.0   14
SF Christian McCaffrey (MVP) 39.22709035581182 9.582500189826602 19500   14


In [159]:
# Create upload CSV by reordering columns to match template order.

headers = []
players = []
ORDERED_COLS = ['QB','RB','RB','WR','WR','WR','TE','FLEX','DEF'] # template order.

def get_match_names(col):
    if col == 'DEF':
        return ['D']
    elif col == 'FLEX' :
        return ['RB', 'WR']
    return [col]

roster_copy = roster.players.copy()
for c in ORDERED_COLS:
    headers.append(c)
    match_names = get_match_names(c)
    for r in roster_copy:
        if r.pos in match_names:
            p = f"{r.kv_store['Id']}:{r.name}"
            players.append(p)
            roster_copy.remove(r)
            break

with open(UPLOAD_FILE, 'w') as f:
    f.write(','.join(headers))
    f.write('\n')
    f.write(','.join(players))

print('done')


done


In [160]:
output = pd.read_csv(UPLOAD_FILE)
output

,QB,RB,RB.1,WR,WR.1,WR.2,TE,FLEX,DEF
0,124667-89675:Jonathan Taylor,124667-90562:Brian Robinson Jr.,124667-61593:Jauan Jennings,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
m_score = df[(df['FPPG'] > 0) & (df['Played'] > 0)].groupby(['Position'])['FPPG'].mean().to_dict()
m_score['DEF'] = m_score['D']
m_score['FLEX'] = (m_score['WR']+m_score['RB'])/2
m_score

{'D': 5.607142857142858,
 'K': 9.171328671328672,
 'QB': 11.77433312733968,
 'RB': 9.582500189826602,
 'TE': 7.763708532630623,
 'WR': 6.645921938256358,
 'DEF': 5.607142857142858,
 'FLEX': 8.11421106404148}